In [ ]:
# !pip install nltk -q

In [ ]:
import pandas as pd
import string
from nltk import ngrams, tokenize, corpus
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stopwords = corpus.stopwords.words('finnish') + list(string.punctuation)

In [ ]:
df = pd.read_csv('data/interim/row_df_fi.csv', index_col=0)
df.head()

In [ ]:
df['organisaatio1'].value_counts()

In [ ]:
stemmer = SnowballStemmer('finnish')
tokenizer = tokenize.WordPunctTokenizer()

stopword_stems = set([stemmer.stem(s) for s in stopwords])

In [ ]:
def preprocess(doc, stem=True):
    stems = [stemmer.stem(token).lower() if stem else token.lower() for token in tokenizer.tokenize(doc)]
    
    grams = list(ngrams(stems, 1)) + list(ngrams(stems, 2))
    
    sw = stopword_stems if stem else stopwords

    grams = [ng for ng in grams if not any(elem in sw or elem in string.punctuation for elem in ng)]
    
    return grams

In [ ]:
sel = df['organisaatio1'] == 'Kasvatuksen ja koulutuksen toimiala'
sel_answers = df.loc[sel, 'answer']

sel_osaamiset = pd.Series([ng for feature_list in sel_answers.apply(preprocess) for ng in feature_list]).value_counts()

for k, v in list(zip(sel_osaamiset.index, sel_osaamiset.values))[20:]:
    print(k, v)
    if v < 10:
        break